In [1]:
import os, sys
import numpy as np
import json
import random

## Prepare the data --> one hot encoding matrices

In [2]:
aptamer_dataset_file = "../data/aptamer_dataset.json"

def construct_dataset():
    with open(aptamer_dataset_file, 'r') as f:
        aptamer_data = json.load(f)
    full_dataset = []
    for aptamer in aptamer_data:
        peptides = aptamer_data[aptamer]
        for peptide, _ in peptides:
            if len(aptamer) == 40 and len(peptide) == 8:
                full_dataset.append((aptamer, peptide))
    return full_dataset

In [3]:
full_dataset = construct_dataset()
random.shuffle(full_dataset)
training_set = full_dataset[:int(0.8*len(full_dataset))]
test_set = full_dataset[int(0.8*len(full_dataset)):]

In [4]:
aa_list = ['R', 'L', 'S', 'A', 'G', 'P', 'T', 'V', 'N', 'D', 'C', 'Q', 'E', 'H', 'I', 'K', 'M', 'F', 'W', 'Y']
na_list = ['A', 'C', 'G', 'T']

## Takes a peptide and aptamer sequence and converts to one-hot matrix
def one_hot(sequence, seq_type='peptide'):
    if seq_type == 'peptide':
        letters = aa_list
    else:
        letters = na_list
    one_hot = np.zeros((len(sequence), len(letters)))
    for i in range(len(sequence)):
        element = sequence[i]
        idx = letters.index(element)
        one_hot[i][idx] = 1
    return one_hot

## Model --> CNN

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD

In [41]:
class TwoLayer(nn.Module):
    def __init__(self):
        super(TwoLayer, self).__init__()
        self.linear_apt_1 = nn.Linear(40, 40)
        self.linear_apt_2 = nn.Linear(40, 1)
        
        self.linear_pep_1 = nn.Linear(8, 8)
        self.linear_pep_2 = nn.Linear(8, 1)
        
        self.relu = nn.ReLU()
    
        self.sequential_pep = nn.Sequential(self.linear_pep_1,
                                            self.relu,
                                            self.linear_pep_2)
        
        self.sequential_apt = nn.Sequential(self.linear_apt_1,
                                            self.relu,
                                            self.linear_apt_2)
                
    def forward(self, apt, pep):
        apt = self.sequential_apt(apt)
        pep = self.sequential_pep(pep)
        print(apt.shape())
        apt = apt.view(-1, 1).T
        pep = pep.view(-1, 1).T
        x = torch.cat((apt, pep), 1)
        x = F.sigmoid(x)
        return x
    
    def loss(self, prediction, label):
        l = nn.MSELoss()
        label = torch.FloatTensor(label)
        label = label.reshape((1, 1))
        return l(torch.FloatTensor(prediction), label)

In [19]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.cnn_apt_1 = nn.Conv2d(1, 40, 1)
        self.cnn_apt_2 = nn.Conv2d(40, 10, 1)
        self.cnn_apt_3 = nn.Conv2d(10, 1, 1)
        self.fc_apt_1 = nn.Linear(160, 1)
        
        self.cnn_pep_1 = nn.Conv2d(1, 8, 1)
        self.cnn_pep_2 = nn.Conv2d(8, 1, 1)
        self.fc_pep_1 = nn.Linear(64, 1)
        
        self.pool = nn.MaxPool2d(2, 1)
        self.relu = nn.ReLU()
        
        #self.dropout = nn.Dropout(0.1)
        
        self.sequential_pep = nn.Sequential(self.cnn_pep_1,
                                            #self.dropout,
                                            self.relu, 
                                            self.pool, 
                                            self.cnn_pep_2)
        
        self.sequential_apt = nn.Sequential(self.cnn_apt_1, 
                                            #self.dropout,
                                            self.relu, 
                                            self.pool, 
                                            self.cnn_apt_2, 
                                            #self.dropout,
                                            self.relu, 
                                            self.pool, 
                                            self.cnn_apt_3)
        
        self.fc1 = nn.Linear(209, 1)
        
    def forward(self, apt, pep):
        apt = self.sequential_apt(apt)
        pep = self.sequential_pep(pep)
        
        apt = apt.view(-1, 1).T
        pep = pep.view(-1, 1).T
        
        x = torch.cat((apt, pep), 1)
        x = self.fc1(x)
        x = F.sigmoid(x)
        return x
    
    def loss(self, prediction, label):
        l = nn.MSELoss()
        label = torch.FloatTensor(label)
        label = label.reshape((1, 1))
        return l(torch.FloatTensor(prediction), label)

In [26]:
model = ConvNet()
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_uniform_(m.weight.data)
        nn.init.zeros_(m.bias.data)
    if isinstance(m, nn.Linear):
        nn.init.kaiming_uniform_(m.weight.data, nonlinearity='relu')
        nn.init.zeros_(m.bias.data)

#model_2 = TwoLayer()
# def weights_init_2(m):
#     if isinstance(m, nn.Linear):
#         nn.init.xavier_uniform_(m.weight.data)
#         nn.init.zeros_(m.bias.data)

model.apply(weights_init)
optimizer = Adam(model.parameters(), lr=1e-7, weight_decay=1e-5)

In [27]:
# Training Loop
import tqdm
for epoch in range(1):
    print("Epoch: ", epoch)
    model.train()
    running_loss = 0.0
    # Come up with a trainloader
    for i, data in enumerate(tqdm.tqdm(training_set[:10000])):
        # Peptide and aptamer, one-hot encode them
        pep = training_set[i][1]
        apt = training_set[i][0]
        
        pep = one_hot(pep, seq_type='peptide')
        apt = one_hot(apt, seq_type='aptamer')
        
        pep = torch.FloatTensor(np.reshape(pep, (1, 1, pep.shape[0], pep.shape[1])))
        apt = torch.FloatTensor(np.reshape(apt, (1, 1, apt.shape[0], apt.shape[1])))
        
        output = model(apt, pep)
        loss = model.loss(output, 1)
        optimizer.zero_grad()
        loss.backward()
        
        #hyperparameter
        clip = 5
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        running_loss += loss.item()
        if i % 500 == 499:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 500))
            running_loss = 0.0
    
print('Finished Training')



  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 1/10000 [00:00<21:58,  7.58it/s]

Epoch:  0




  0%|          | 8/10000 [00:00<16:05, 10.35it/s]

  0%|          | 16/10000 [00:00<11:55, 13.96it/s]

  0%|          | 24/10000 [00:00<09:00, 18.46it/s]

  0%|          | 33/10000 [00:00<06:51, 24.23it/s]

  0%|          | 43/10000 [00:00<05:20, 31.05it/s]

  1%|          | 53/10000 [00:00<04:17, 38.60it/s]

  1%|          | 61/10000 [00:00<03:49, 43.36it/s]

  1%|          | 70/10000 [00:01<03:15, 50.84it/s]

  1%|          | 78/10000 [00:01<02:53, 57.07it/s]

  1%|          | 86/10000 [00:01<02:44, 60.34it/s]

  1%|          | 95/10000 [00:01<02:28, 66.60it/s]

  1%|          | 104/10000 [00:01<02:18, 71.26it/s]

  1%|          | 114/10000 [00:01<02:09, 76.50it/s]

  1%|          | 124/10000 [00:01<02:05, 78.84it/s]

  1%|▏         | 134/10000 [00:01<01:58, 83.00it/s]

  1%|▏         | 143/10000 [00:01<01:56, 84.62it/s]

  2%|▏         | 152/10000 [00:01<01:55, 85.61it/s]

  2%|▏         | 161/10000 [00:02<01:56, 84.72it/s]

  2%|▏         | 170/10000 [00:02<01:54, 85.84it/s]

  2

[1,   500] loss: 0.541




  5%|▌         | 521/10000 [00:06<01:55, 82.06it/s]

  5%|▌         | 532/10000 [00:06<01:48, 87.26it/s]

  5%|▌         | 542/10000 [00:06<01:46, 88.99it/s]

  6%|▌         | 551/10000 [00:06<01:48, 86.95it/s]

  6%|▌         | 561/10000 [00:06<01:46, 88.56it/s]

  6%|▌         | 571/10000 [00:06<01:44, 89.90it/s]

  6%|▌         | 581/10000 [00:07<01:43, 90.65it/s]

  6%|▌         | 591/10000 [00:07<01:48, 86.43it/s]

  6%|▌         | 600/10000 [00:07<01:56, 80.49it/s]

  6%|▌         | 609/10000 [00:07<02:00, 77.95it/s]

  6%|▌         | 617/10000 [00:07<02:03, 76.15it/s]

  6%|▋         | 625/10000 [00:07<02:05, 74.88it/s]

  6%|▋         | 633/10000 [00:07<02:07, 73.57it/s]

  6%|▋         | 641/10000 [00:07<02:08, 72.89it/s]

  6%|▋         | 649/10000 [00:08<02:07, 73.58it/s]

  7%|▋         | 658/10000 [00:08<02:04, 75.31it/s]

  7%|▋         | 666/10000 [00:08<02:03, 75.62it/s]

  7%|▋         | 674/10000 [00:08<02:03, 75.56it/s]

  7%|▋         | 683/10000 [00:08<01:58, 78.

...

[1,  9000] loss: 0.495




 90%|█████████ | 9020/10000 [01:52<00:13, 70.59it/s]

 90%|█████████ | 9028/10000 [01:53<00:13, 70.49it/s]

 90%|█████████ | 9037/10000 [01:53<00:13, 73.75it/s]

 90%|█████████ | 9046/10000 [01:53<00:12, 77.32it/s]

 91%|█████████ | 9055/10000 [01:53<00:11, 79.32it/s]

 91%|█████████ | 9064/10000 [01:53<00:11, 81.01it/s]

 91%|█████████ | 9073/10000 [01:53<00:11, 81.84it/s]

 91%|█████████ | 9082/10000 [01:53<00:11, 77.88it/s]

 91%|█████████ | 9090/10000 [01:53<00:12, 75.40it/s]

 91%|█████████ | 9099/10000 [01:53<00:11, 78.42it/s]

 91%|█████████ | 9107/10000 [01:54<00:11, 78.62it/s]

 91%|█████████ | 9115/10000 [01:54<00:11, 78.24it/s]

 91%|█████████ | 9124/10000 [01:54<00:10, 79.86it/s]

 91%|█████████▏| 9134/10000 [01:54<00:10, 84.55it/s]

 91%|█████████▏| 9143/10000 [01:54<00:10, 84.91it/s]

 92%|█████████▏| 9153/10000 [01:54<00:09, 88.25it/s]

 92%|█████████▏| 9162/10000 [01:54<00:09, 88.01it/s]

 92%|█████████▏| 9171/10000 [01:54<00:09, 88.20it/s]

 92%|█████████▏| 9180/1000

[1,  9500] loss: 0.486




 95%|█████████▌| 9519/10000 [01:58<00:05, 93.47it/s]

 95%|█████████▌| 9529/10000 [01:58<00:05, 90.52it/s]

 95%|█████████▌| 9539/10000 [01:59<00:05, 85.05it/s]

 95%|█████████▌| 9548/10000 [01:59<00:05, 84.47it/s]

 96%|█████████▌| 9557/10000 [01:59<00:05, 86.00it/s]

 96%|█████████▌| 9567/10000 [01:59<00:04, 87.65it/s]

 96%|█████████▌| 9576/10000 [01:59<00:04, 87.61it/s]

 96%|█████████▌| 9586/10000 [01:59<00:04, 89.95it/s]

 96%|█████████▌| 9596/10000 [01:59<00:04, 91.21it/s]

 96%|█████████▌| 9606/10000 [01:59<00:04, 91.96it/s]

 96%|█████████▌| 9616/10000 [01:59<00:04, 91.51it/s]

 96%|█████████▋| 9626/10000 [01:59<00:04, 90.13it/s]

 96%|█████████▋| 9637/10000 [02:00<00:03, 94.21it/s]

 96%|█████████▋| 9647/10000 [02:00<00:03, 90.45it/s]

 97%|█████████▋| 9657/10000 [02:00<00:03, 86.43it/s]

 97%|█████████▋| 9666/10000 [02:00<00:03, 83.95it/s]

 97%|█████████▋| 9675/10000 [02:00<00:03, 82.18it/s]

 97%|█████████▋| 9685/10000 [02:00<00:03, 85.18it/s]

 97%|█████████▋| 9695/1000

[1, 10000] loss: 0.513
Finished Training


## Evaluation --> compare to random

In [28]:
correct = 0
incorrect = 0
for i, data in enumerate(tqdm.tqdm(test_set[:10000])):
    pep = test_set[i][1]
    apt = test_set[i][0]
    
    pep = one_hot(pep, seq_type='peptide')
    apt = one_hot(apt, seq_type='aptamer')

    pep = torch.FloatTensor(np.reshape(pep, (1, 1, pep.shape[0], pep.shape[1])))
    apt = torch.FloatTensor(np.reshape(apt, (1, 1, apt.shape[0], apt.shape[1])))

    output = model(apt, pep)
    #print("Output: ", output)
    
    if output > 0.5:
        correct += 1
    else:
        incorrect += 1

print('Accuracy of the network on the test samples: %d %%' % (100* correct/(correct + incorrect)))



  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 45/10000 [00:00<00:22, 448.78it/s]

  1%|          | 94/10000 [00:00<00:21, 458.87it/s]

  1%|▏         | 143/10000 [00:00<00:21, 467.20it/s]

  2%|▏         | 192/10000 [00:00<00:20, 473.48it/s]

  2%|▏         | 241/10000 [00:00<00:20, 478.31it/s]

  3%|▎         | 290/10000 [00:00<00:20, 481.48it/s]

  3%|▎         | 339/10000 [00:00<00:20, 481.56it/s]

  4%|▍         | 388/10000 [00:00<00:19, 483.10it/s]

  4%|▍         | 437/10000 [00:00<00:19, 483.83it/s]

  5%|▍         | 486/10000 [00:01<00:19, 484.72it/s]

  5%|▌         | 535/10000 [00:01<00:19, 485.31it/s]

  6%|▌         | 584/10000 [00:01<00:19, 485.30it/s]

  6%|▋         | 634/10000 [00:01<00:19, 487.05it/s]

  7%|▋         | 684/10000 [00:01<00:19, 489.30it/s]

  7%|▋         | 734/10000 [00:01<00:18, 491.42it/s]

  8%|▊         | 784/10000 [00:01<00:18, 493.59it/s]

  8%|▊         | 834/10000 [00:01<00:18, 495.25it/s]

  9%|▉         | 884/10000 [00:01<00:18

 90%|█████████ | 9039/10000 [00:13<00:01, 499.33it/s]

 91%|█████████ | 9089/10000 [00:13<00:01, 499.43it/s]

 91%|█████████▏| 9139/10000 [00:13<00:01, 499.47it/s]

 92%|█████████▏| 9189/10000 [00:14<00:01, 499.37it/s]

 92%|█████████▏| 9239/10000 [00:14<00:01, 499.43it/s]

 93%|█████████▎| 9289/10000 [00:14<00:01, 499.21it/s]

 93%|█████████▎| 9339/10000 [00:14<00:01, 499.17it/s]

 94%|█████████▍| 9389/10000 [00:14<00:01, 499.00it/s]

 94%|█████████▍| 9439/10000 [00:14<00:01, 499.03it/s]

 95%|█████████▍| 9490/10000 [00:14<00:01, 499.40it/s]

 95%|█████████▌| 9540/10000 [00:14<00:00, 499.23it/s]

 96%|█████████▌| 9590/10000 [00:14<00:00, 499.26it/s]

 96%|█████████▋| 9640/10000 [00:14<00:00, 499.18it/s]

 97%|█████████▋| 9690/10000 [00:15<00:00, 499.19it/s]

 97%|█████████▋| 9740/10000 [00:15<00:00, 499.39it/s]

 98%|█████████▊| 9790/10000 [00:15<00:00, 499.22it/s]

 98%|█████████▊| 9840/10000 [00:15<00:00, 499.15it/s]

 99%|█████████▉| 9890/10000 [00:15<00:00, 498.76it/s]

 99%|█████

Accuracy of the network on the test samples: 65 %


# sampling --> need to change

In [29]:
k = 1e5

# Generate uniformly without replacement
def get_samples(kind="pep",num=k):
    if kind == "apt":
        samples = [all_aptamers[i] for i in np.random.choice(len(all_aptamers), num_samples, replace=False)]
    else:
        samples = [all_peptides[i] for i in np.random.choice(len(all_peptides), num_samples, replace=False)]
    return samples

# Sample x' from P_X (assume peptides follow NNK)
def get_x_prime(k):
    x_primes = []
    for _ in range(k):
        pvals = [0.089]*3 + [0.065]*5 + [0.034]*12
        x_idx = np.random.choice(20, 7, p=pvals)
        x_prime = "M"
        for i in x_idx:
            x_prime += aa_list[i]
        x_primes.append(x_prime)
    return x_primes

# Sample y' from P_Y (assume apatamers follow uniform)
def get_y_prime(k):
    y_primes = []
    for _ in range(k):
        y_idx = np.random.randint(0, 4, 40)
        y_prime = ""
        for i in y_idx:
            y_prime += na_list[i]
        y_primes.append(y_prime)
    return y_primes